In [ ]:
import time
import numpy as np

In [ ]:
import sc3nb as scn

# OSC communication
With the OSC communication module of sc3nb you can directly send and receive OSC packets.

Open Sound Control (OSC) is a networking protocol for sound and is used by SuperCollider to communicate between sclang and scsynth. sc3nb is itself a OSC client and server. This allows sc3nb to send and receive OSC traffic.

For more information on OSC and especially how Supercollider handles OSC packets please refer to the following links:

* [Open Sound Control Specification](http://opensoundcontrol.org/spec-1_0)
* [Server vs Client SuperCollider Guide](http://doc.sccode.org/Guides/ClientVsServer.html)
* [Server Command Reference](https://doc.sccode.org/Reference/Server-Command-Reference.html)
* [SuperCollider Synth Server Architecture](https://doc.sccode.org/Reference/Server-Architecture.html)

In [ ]:
sc = scn.startup()

sc3nb serves as OSC server and as client of the SuperCollider server `scsynth`.<br>
You can also communicate with the SuperCollider interpreter `sclang` via OSC.<br>


You can see the current connection information with `sc.server.connection_info()`

In [ ]:
(sc3nb_ip, sc3nb_port), receivers = sc.server.connection_info()

In [ ]:
(sc3nb_ip, sc3nb_port), receivers

If you want to communicate via OSC with another receiver you could add it's name via `sc.server.add_receiver(name: str, ip: str, port: int)` or you can pass a custom receiver when sending OSC

In [ ]:
sc.server.add_receiver("sc3nb", sc3nb_ip, sc3nb_port)

In [ ]:
sc.server.connection_info()

## Sending OSC

You can send OSC with 

In [ ]:
help(sc.server.send)

### Messages

Use the `OSCMessage` or the `python-osc` package to build a OscMessage

In [ ]:
scn.OSCMessage?

In [ ]:
msg = scn.OSCMessage("/s_new", ["s1", -1, 1, 1,])
sc.server.send(msg)

A shortcut for sending Messages is

In [ ]:
help(sc.server.msg)

In [ ]:
sc.server.msg("/s_new", ["s1", -1, 1, 1,])

a more complex example

In [ ]:
for p in [0,2,4,7,5,5,9,7,7,12,11,12,7,4,0,2,4,5,7,9,7,5,4,2,4,0,-1,0,2,-5,-1,2,5,4,2,4]:
    freq = scn.midicps(60+p)  # see helper fns below
    sc.server.msg("/s_new", ["s1", -1, 1, 0, "freq", freq, "dur", 0.5, "num", 1])
    time.sleep(0.15)

**Note** that the timing is here under python's control, which is not very precise. Here comes the `Bundler` class into play.

**Remarks**:
* note that the python code returns immediately and all events remain in scsynth
* note that unfortunately scsynth has a limited buffer for OSC messages, so it is not viable to spawn thousends of events. scsynth will then simply reject OSC messages.
* this motivated (and is solved) with a TimedQueue, see below.

### Bundles

In [ ]:
from sc3nb.osc.osc_communication import Bundler

To send one or multiple message(s) with a timetag as a OSC Bundle you should use the `Bundler` class

* Bundlers allow to specify a timetag and thus let scsynth control the timing, which is much better, if applicable.
* Bundler can be created by


In [ ]:
Bundler?

To add messages to the Bundler use 

The prefered way of creating Bundlers for sending to the server is via

In [ ]:
help(sc.server.bundler)

This will add the `sc.server.latency` time to the timetag. By default this is `0.0` but you can set it.

In [ ]:
sc.server.latency

In [ ]:
sc.server.latency = 0.1
sc.server.latency

A Bundler lets you add Messages with

In [ ]:
help(Bundler.add)

In [ ]:
msg1 = scn.OSCMessage("/s_new", ["s2", -1, 1, 1,])
msg2 = scn.OSCMessage("/n_free", [-1])
sc.server.bundler().add(msg1).add(0.4, msg2).send()

Simpler is the usage of the context manager:


This means you can use `with` for better handling 

In [ ]:
with sc.server.bundler() as bundler:
    bundler.add(0.0, msg1)
    bundler.add(0.3, msg2)

Here are some different styles of writing the same Sound with the Bundler features 

 - server Bundler with add

In [ ]:
with sc.server.bundler(send_on_exit=False) as bundler:
    bundler.add(0.0, "/s_new", ["s2", -1, 1, 1,])
    bundler.add(0.3, "/n_free", [-1])

In [ ]:
dg1 = bundler.to_raw_osc(0.0)  # we set the time_offset explicitly so all Bundle datagrams are the same
dg1

 - Bundler with explict latency set and using add

In [ ]:
with Bundler(sc.server.latency, send_on_exit=False) as bundler:
    bundler.add(0.0, "/s_new", ["s2", -1, 1, 1])
    bundler.add(0.3, "/n_free", [-1])

In [ ]:
dg2 = bundler.to_raw_osc(0.0)
dg2

 - server Bundler with implicit latency and using bundled messages

In [ ]:
with sc.server.bundler(send_on_exit=False) as bundler:
    sc.server.msg("/s_new", ["s2", -1, 1, 1,], bundle=True)
    bundler.wait(0.3)
    sc.server.msg("/n_free", [-1], bundle=True)

In [ ]:
dg3 = bundler.to_raw_osc(0.0)
dg3

In [ ]:
assert dg1 == dg2 and dg1 == dg2, "The datagrams are not the same"

Note that you can use the Bundler with the Synth and Group for easier Message creation.

Also make sure to look at the [Automatic Bundling Feature](#Automatic-Bundling) which is using the bundled messages (`msg(..., bundle=True`)

In [ ]:
t0 = time.time()
with sc.server.bundler() as bundler:
    for i, r in enumerate(np.random.randn(10)):
        onset = t0 + 3 + r
        freq = 500 + 5 * i
        bundler.add(onset, scn.Synth("s1", {"freq": freq, "dur": 1.5, "num": abs(r)+1}, new=False).new(return_msg=True))

#### Bundler Timestamp

small numbers are times in seconds relative to `time.time()` evaluated at the time of sending

In [ ]:
sc.server.bundler(0.5, "/s_new", ["s1", -1, 1, 0, "freq", 200, "dur", 1]).send()  # a tone starts in 0.5s
sc.server.bundler(1.0, "/s_new", ["s1", -1, 1, 0, "freq", 300, "dur", 1]).send()  # a tone starts in 1.0s

**Attention:** 

Sending bundles with relative times could lead to unprecise timings.
If you care about precise timings you should

* use one bundler with multiple messages (if you care about the timings relative to each other in one Bundler)
    * because all relative times of the inner messages are calculated on top of the outermost bundler timetag
* or provide a explict timetag (>1e6) to specify absolute times (see next examples)


One Bundler with multiple messages

In [ ]:
bundler = sc.server.bundler()
bundler.add(0.5, "/s_new", ["s1", -1, 1, 0, "freq", 200, "dur", 1])
bundler.add(1.0, "/s_new", ["s1", -1, 1, 0, "freq", 300, "dur", 1])
bundler.send()  # a tone starts in 1.0s

using `time.time()+timeoffset` for absolute times

In [ ]:
t0 = time.time()
sc.server.bundler(t0 + 0.5, "/s_new", ["s1", -1, 1, 0, "freq", 200, "dur", 1]).send()  # a tone starts in 0.5s
sc.server.bundler(t0 + 1.0, "/s_new", ["s1", -1, 1, 0, "freq", 300, "dur", 1]).send()  # a tone starts in 1.0s

In [ ]:
t0 = time.time()
with sc.server.bundler() as bundler:
    for i, r in enumerate(np.random.randn(100)):
        onset = t0 + 3 + r
        freq = 500 + 5 * i
        msg_params = ["s1", -1, 1, 0, "freq", freq, "dur", 1.5, "num", abs(r)+1]
        bundler.add(onset, "/s_new", msg_params)

#### Nesting Bundlers

You can even nest bundler. This will recalculate the time relative to the sending time of the outermost bundler

In [ ]:
with sc.server.bundler() as bundler_outer:
    with sc.server.bundler() as bundler:
        sc.server.msg("/s_new", ["s2", 3333, 1, 1,], bundle=True)
        bundler.wait(0.3)
        sc.server.msg("/n_free", [3333], bundle=True)
    bundler_outer.wait(0.4)
    bundler_outer.add(bundler)

### Automatic Bundling

Probably the most convenient way of sending OSC is by using the Automatic Bundling Feature.

This allows you to simply use the SuperCollider Objects Synth and Group in the Context Manager of a Bundler and they will be automatically captured and stored. 

In [ ]:
with sc.server.bundler() as bundler:
    synth = scn.Synth("s2")
    bundler.wait(0.3)
    synth.set("freq", 1000)
    bundler.wait(0.1)
    synth.free()
synth.wait()

**Note** that it is important that we only wait on the Synth after the context of the Bundler is closed.

If you would call `synth.wait()` in the Bundler context it would wait before sending the /s_new Message to the server and then wait forever (or until timeout) for the /n_end notification.

In [ ]:
try:
    with sc.server.bundler() as bundler:
        synth = scn.Synth("s2")
        bundler.wait(0.3)
        synth.set("freq", 1000)
        bundler.wait(0.1)
        synth.free()
        synth.wait(timeout=2)  # without a timeout this would hang forever
except RuntimeError as error:
    print(error)

## Receiving OSC

sc3nb is receiving OSC messages with the help of queues, for each OSC address where we want to receive messages there is a AddressQueue

In [ ]:
sc.server.msg_queues

To see more information what messages are send and received set the logging level to INFO as seen below.

In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
# even more verbose logging is also avaible
# logging.basicConfig(level=logging.DEBUG)

### Getting replies

For some outgoing Messages there is a incoming Message defined.

This means sending such a message will automatically wait for the incoming Message at the corresponding Queue and return the result.

An example for this is `/sync {sync_id} -> /synced {sync_id}` 

In [ ]:
sc.server.msg("/sync", 12345)

See all `(outgoing message, incoming message)` pairs:

In [ ]:
sc.server.reply_addresses

You can get the reply address via

In [ ]:
sc.server.get_reply_address("/sync")

or

In [ ]:
sc.server.reply_addresses["/sync"]

If we specify `await_reply=False` the message will be kept in the queue

In [ ]:
sc.server.msg("/sync", 1, await_reply=False)

In [ ]:
sc.server.msg_queues[sc.server.get_reply_address("/sync")]

In [ ]:
sc.server.msg("/sync", 2, await_reply=False)

In [ ]:
sc.server.msg_queues[sc.server.reply_addresses["/sync"]]

In [ ]:
sc.server.msg_queues["/synced"]

You can see how many values were hold.

In [ ]:
sc.server.msg_queues["/synced"].skips

Notice that these hold messages will be skipped. 

In [ ]:
sc.server.msg("/sync", 3, await_reply=True)

In [ ]:
sc.server.msg_queues["/synced"]

Therefore you should retrieve them with `get` and set `skip=False` if you care for old values in the queue and dont want them to be skipped.

In [ ]:
sc.server.msg("/sync", 42, await_reply=False)
sc.server.msg_queues["/synced"].get(skip=False)

### Custom Message Queues

If you want to get additional OSC Messages you need to create a custom MessageQueue

In [ ]:
from sc3nb.osc.osc_communication import MessageQueue

In [ ]:
help(MessageQueue)

In [ ]:
mq = MessageQueue("/test")

In [ ]:
sc.server.add_msg_queue(mq)

In [ ]:
sc.server.msg("/test", ["Hi!"], receiver="sc3nb")

In [ ]:
sc.server.msg_queues["/test"]

In [ ]:
sc.server.msg("/test", ["Hello!"], receiver="sc3nb")

In [ ]:
sc.server.msg_queues["/test"]

In [ ]:
sc.server.msg_queues["/test"].get()

In [ ]:
sc.server.msg_queues["/test"].get()

If you want to create a pair of a outgoing message that will receive a certain incomming message you need to specify it via the `out_addr` arugment of `add_msg_queue` or you could use the shortcut for this `add_msg_pairs`

In [ ]:
help(sc.server.add_msg_pairs)

In [ ]:
sc.server.add_msg_pairs({"/hi": "/hi.reply"})

Lets use `OSCdef` in sclang to send us replies.

In [ ]:
%%scv
OSCdef.newMatching("say_hi", {|msg, time, addr, recvPort| addr.sendMsg("/hi.reply", "Hello there!")}, '/hi');

In [ ]:
sc.server.msg("/hi", receiver="sclang")

There is also the class MessageQueueCollection, this allows to create multiple MessageQueues for a multiple `addess/subaddresses` combination

In [ ]:
from sc3nb.osc.osc_communication import MessageQueueCollection

In [ ]:
help(MessageQueueCollection)

In [ ]:
mqc = MessageQueueCollection("/collect", ["/address1", "/address2"])
sc.server.add_msg_queue_collection(mqc)

In [ ]:
mqc = MessageQueueCollection("/auto_collect")
sc.server.add_msg_queue_collection(mqc)

In [ ]:
sc.server.reply_addresses

In [ ]:
sc.server.msg_queues

In [ ]:
%%scv
OSCdef.newMatching("ab", {|msg, time, addr, recvPort| addr.sendMsg('/collect', '/address1', "toast".scramble)}, '/address1');

In [ ]:
%%scv
OSCdef.newMatching("ab", {|msg, time, addr, recvPort| addr.sendMsg('/collect', '/address2', "sonification".scramble)}, '/address2');

In [ ]:
sc.server.msg("/address1", receiver="sclang")

In [ ]:
sc.server.msg("/address2", receiver="sclang")

## Examples 

### Creating an OSC responder and msg to sclang for synthesis

In [ ]:
%%scv
OSCdef(\dinger, { | msg, time, addr, recvPort |
    var freq = msg[2];
    {Pulse.ar(freq, 0.04, 0.3)!2 * EnvGen.ar(Env.perc, doneAction:2)}.play()
}, '/ding')

In [ ]:
with scn.Bundler(receiver=sc.lang.addr):
    for i in range(5):
        sc.server.msg("/ding", ["freq", 1000-5*i], bundle=True)

In [ ]:
for i in range(5):
    sc.server.msg("/ding", ["freq", 1000-5*i], receiver=sc.lang.addr)

In [ ]:
%scv OSCdef.freeAll()

In [ ]:
sc.exit()